# Part 1 - Planning an experiment

In this part, we will first start to think of how to perform an fMRI experiment. First, think of a controlled experiment that you would like to do, and how you can use two conditions to study the brain. For example, if you are interested in color representation, you could show images that are colored in one condition, and images that are black and white in the other. To find where color is represented, you can do a contrast in which you see which regions have higher activity when looking at colored images versus black and white images.

(You have to chose another example).

## 1.1 Designing the experiment

In the space below, describe an experiment that you would like to test:
- Describe the question you are interested in.
- What are the two conditions you would use? Make sure that your conditions allow you to answer the question you asked.
- Describe a simple experiment that you would run: what images, sounds, or other stimuli would you use? how long would the experiment be? what would the subjects be doing?

In [ ]:
### STUDENT ANSWER


## 1.2 Analyzing the data

Now assume that you have indeed collected this data and preprocessed it. It is now loaded on your computer in an array format, and so is the design matrix that describes the order of appearance of your stimulus. What are the steps that are required for you to find out which brain regions are more activated in your task of interest? Write up all the analytical steps you need to take. We are not asking you to write code, only a description of the steps that you need to do.

In [ ]:
### STUDENT ANSWER


# Part 2 - Motor Localizer

In this part, we will look at another localizer dataset, in which subjects performed different motor actions in the scanner. We load the data here and look at the conditions:

In [ ]:
# Imports
import neurods as nds
import numpy as np
import h5py, os
import matplotlib.pyplot as plt
# Configure defaults for plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.aspect'] = 'auto'
plt.rcParams['image.cmap'] = 'viridis'
%matplotlib inline

In [ ]:
# Loading the design
# (hdf files are a standard data storage format; 
# here, we simply load data to arrays and lists)
basedir = os.path.join(nds.io.data_list['fmri'],'motor')
design = h5py.File(os.path.join(basedir,'design_motor.hdf'))
conditions = [s.astype('str') for s in design['xnames'][:]]
print('Conditions: ', conditions)
X = design['X'][:]

plt.figure(figsize=(10,4))
for i, (cond, label) in enumerate(zip(X.T, conditions)):
    plt.plot(cond+i+0.2*i, label=label, lw=2)
plt.title('Condition labels')
_ = plt.legend(frameon=False, bbox_to_anchor=(1.4, 1))

## 2.1 Describing the experiment

- Describe the above experiment in two sentences. 
- What was the length of each condition?
- What do you think you can test about the brain using this data?

In [ ]:
### STUDENT ANSWER


## 2.2 Loading the data and normalizing it

Here we load the data. 

In [ ]:
import cortex
sub, xfm = 'S3', 's03_motorloc'

mask = cortex.db.get_mask(sub, xfm, type='cortical')
fname = os.path.join(basedir, 's03_motorloc.nii.gz')
# fmri responses:
Y = nds.fmri.load_data(fname, mask=mask)

plt.figure(figsize=(10,4))
plt.imshow(Y)
plt.title('Voxel responses');

- print the shape of the brain data, now called "Y" and of the design matrix data, now called "X". 
- use the zscore function imported below to standardize the **columns** of "Y" and "X". 
- What is now the mean and standard deviation of the columns of Y and X? The best way to show this to plot the mean and standard deviation of the columns of Y and X.

In [ ]:
from scipy.stats import zscore
### STUDENT ANSWER


## 2.3 Accounting for the hemodynamic response

We saw in the lectures that we need to account for the delay in the hemodynamic response. We first use the neurods.fmri function that allows us to generate a canonical hemodynamic response function (hrf):

- knowing that this localizer was scanned with a TR of 2 seconds, use the generate_hrf function below to estimate a canonical hrf.
- Plot the canonical hrf.

In [ ]:
from neurods.fmri import hrf as generate_hrf
### STUDENT ANSWER


- Now use the hrf you estimated above, and the np.convolve function, to create conv_X, a version of the design matrix that accounts for the hrf delay. Remember, the np.convolve function returns a signal that is longer than the original signal, so you should trim it appropriately.

In [ ]:
n, d = X.shape

### STUDENT ANSWER


Use the following cell to plot conv_X:

In [ ]:
plt.figure(figsize=(10,4))

for i, (cond, label) in enumerate(zip(conv_X.T, conditions)):
    plt.plot(cond+i+3*i, label=label, lw=2)
    
plt.title('Condition labels')
_ = plt.legend(frameon=False, bbox_to_anchor=(1.4, 1))

## 2.4 Estimating voxel responses to motor actions

In this part, we will estimate the weights of a linear regression model that expresses the activity in a voxel in terms of the experimental conditions.

- implement a function that computes the OLS solution for a given design matrix and a response matrix
- use this function to estimate the weights for all voxels
- print the shape of the estimated weights

In [ ]:
### STUDENT ANSWER


- Now create 6 flatmaps, each for the weights of a condition across all the brain.
    - make sure you title each flatmap with the correct condition name


In [ ]:
# hint: this is how you plot a flatmap: first define a volume with the proper 
# variables, here called "some_vector"
# vol = cortex.Volume(some_vector, sub, xfm, mask = mask)
# __  = cortex.quickflat.make_figure(vol)
# plt.title('some str', fontsize = 30)

### STUDENT ANSWER


## 2.5 Estimating a contrast

We are interested in identifying foot motor regions, and specifically to distinguish them from hand motor regions. 
- Construct a vector **c** that encodes the contrast "**foot - hand**". 
- Use the vector **c** to estimate a contrast between foot and hand.
- Make a flatmap of the contrast.
- Title it appropriately.

In [ ]:
### STUDENT ANSWER


- Describe the plot you see? E.g. which parts of the brain seem to be higher for foot, and for hands? are these regions close to each other?

## 2.6 Estimating a t-statistic

We will use here the t-distribution method we learned in class to estimate a statistic for our contrast. As we saw in the class, we need to first predict the brain activity using our learned weights, then we need to compute the mean squared error (mse). We then can use the vector **c** and our input matrix (**conv_X**) to estimate the t-statistic.

### Predict the activity $\hat Y$

- Using the weight matrix and your conv_X matrix, predict the activity $\hat Y$.

In [ ]:
### STUDENT ANSWER


### Estimating the mse

- Estimate the error $Y-\hat Y$.
- Compute the mean squared error: $\frac{1}{N}\sum_{i=0}^{N-1}(Y_i - \hat Y_i)^2$, i.e. the mse
- Make a flatmap of the mse.
- Which regions have low mean squared errors? What do you think these correspond to?

In [ ]:
### STUDENT ANSWER


### Estimating the t-statistic

Now we can compute the t-statistic:
- use the function below along with the appropriate inputs to estimate a t-statistic for the foot-hand contrast.
- make a flatmap of the resulting statistic.

In [ ]:
def t_stat(X, beta, mse, c):
    num = np.dot(c,weights)
    XtXinv = inv( np.dot(X.T,X) )
    cT_XtXinv_c =  np.dot( np.dot( c, XtXinv),c)
    den = np.sqrt( mse * cT_XtXinv_c )
    return num / den

### STUDENT ANSWER


## 2.7 Estimating a p-value and thresholding for significance

Now that we have a t-statistic, we can estimate a p-value. 

- Use the t-distribution to find the p-value for the t-statistic at each voxel.
- The parameter nu for the t-distribution is set for you.
- Plot a flatmap of p values with a log scale

In [ ]:
from scipy.stats import t as tdistribution
nu = Y.shape[0] - 1

### STUDENT ANSWER


Finally, we can threshold for significance and find the regions that are significantly higher for the "foot" condition than the "hand" condition.

- Starting with a signficance level $\alpha = 0.05$, use Bonferonni correction to obtain $\alpha'$ (you need to use another dimension of the dataset).
- Using $\alpha'$, produce a flatmap of the voxels in which you rejected the null hypothesis, i.e. for which the p-value is smaller than $\alpha'$.

In [ ]:
### STUDENT ANSWER


Do the selected regions make sense according to the flatmap of the contrast you plotted above?

In [ ]:
### STUDENT ANSWER
